In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.enable_eager_execution()


from tensorflow import feature_column
from tensorflow.keras import layers
# tf.disable_v2_behavior()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split

%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
dataframe = pd.read_csv("doginfo.csv")
dataframe.head()

,kindCd,neuterYn,sexCd,weight,noticeDays,age2,processState
0,푸들,N,F,7.46,10,12,0
1,믹스견,N,M,7.00,14,1,1
2,믹스견,U,M,4.50,11,2,0
3,진도견,N,M,10.00,8,1,0
4,믹스견,N,M,6.00,8,4,0


In [7]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), '훈련 샘플')
print(len(val), '검증 샘플')
print(len(test), '테스트 샘플')

14586 훈련 샘플
3647 검증 샘플
4559 테스트 샘플


In [4]:
# 판다스 데이터프레임으로부터 tf.data 데이터셋을 만들기 위한 함수
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('processState')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [5]:
batch_size = 5 # 예제를 위해 작은 배치 크기를 사용합니다.
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [6]:
for feature_batch, label_batch in train_ds.take(1):
  print('전체 특성:', list(feature_batch.keys()))
  print('나이 특성의 배치:', feature_batch['age'])
  print('타깃의 배치:', label_batch )


# 특성 열을 시험해 보기 위해 샘플 배치를 만듭니다.
example_batch = next(iter(train_ds))[0]

# 특성 열을 만들고 배치 데이터를 변환하는 함수
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

전체 특성: ['kindCd', 'neuterYn', 'sexCd', 'weight', 'noticeDays', 'age2']


KeyError: 'age'

In [8]:
# 임베딩 열의 입력은 앞서 만든 범주형 열입니다.
kindCd_embedding = feature_column.embedding_column(kindCd, dimension=114)
demo(kindCd_embedding)

NameError: name 'kindCd' is not defined